In [1]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
from ipywidgets import interact, widgets
from pathlib import Path

from os import listdir
from os.path import isfile, join


p = Path.home()
p = p/'OneDrive'/'Documentos'/'Python'/'Results'/'CSV'/'FTIR'

In [3]:
filenames = [f for f in listdir(p) if isfile(join(p, f))]
frames = dict()
for filename in filenames:
        filepath = p/filename
        try:
            frames[filepath.name] = (pd.read_csv(filepath, skiprows = [0,1,2,3,4]))
            frames[filepath.name].columns = ['Wavenumber','%Transmitance']
        except FileNotFoundError:
            pass

In [4]:
def plot_ftir(key):
    fig  = frames[key].plot(x = 'Wavenumber', y='%Transmitance',kind='line')
    fig.update_layout(xaxis_range = [4000,500])
    return fig

In [5]:
interact(plot_ftir,
         key = list(frames.keys())
        )

interactive(children=(Dropdown(description='key', options=('Grafite-p1-pubgd.csv', 'Grafite-p1.csv', 'Grafite-…

<function __main__.plot_ftir(key)>

In [6]:
absorb = [
    'Grafite-p1-pubgd.csv',
    'Grafite-p1.csv',
    'Grafite-p2-pubgd.csv',
    'Grafite-p2.csv',
    'Misto-p1-pubgd.csv',
    'Misto-p1.csv',
    'Misto-p2-pubgd.csv',
    'Misto-p2.csv',
    'PAni-ATR-Absor-11-s7.csv',
    'PAni-ATR-Trans-12-s2.csv',
    'PAni-ATR-Trans-12-s3.csv',
    'PAni-ATR-Trans-12-s4.csv',
    'Pani-p1-pubgd.csv',
    'Pani-p1.csv',
    'Pani-p2-pubgd.csv',
    'Pani-p2.csv',
    'PAni_antiga.csv',
    'pu-p1.csv',
    'pu-p2.csv',
]
for key in absorb:
    frames[key]['%Transmitance'] *= -1

In [7]:
def normalize(x):
    return np.interp(x,[x.min(), x.max()],[0,1])

In [8]:
for key, frame in frames.items():
    frame.loc[frame['Wavenumber']>=500,'%Transmitance'] = normalize(frame.loc[frame['Wavenumber']>=500,'%Transmitance'])

In [37]:
excluidos = [
    'Grafite-p2.csv',
    'Grafite-p2-pubgd.csv',
    'Grafite_expandido.csv',
    'Grafite_nanoplaquetas.csv',
    'Grafite_natural.csv',
    'Misto-p1-pubgd.csv',
    'Misto-p2.csv',
    'PAni-ATR-Absor-11-s5.csv',
    'PAni-ATR-Absor-11-s7.csv',
    'PAni-ATR-Trans-12-s2.csv',
    'PAni-ATR-Trans-12-s3.csv',
    'PAni-ATR-Trans-12-s4.csv',
    'PAni-NPM.csv',
    'Pani-p1-pubgd.csv',
    'Pani-p2.csv',
    'PAni_antiga.csv',
    'PAni-Po-Absor-12.csv',
    'PAni-Po-Trans-14.csv',
    'PAni-Po-Trans-15.csv',
    'pu-p1.csv',
    'TDI-PAni.csv',
    'Grafite-p1-pubgd.csv',
    'Misto-p2-pubgd.csv',
    'Pani-p2-pubgd.csv',
    'Grafite-p1.csv',
    'Misto-p1.csv',
    'Pani-p1.csv',
    'Iso-PAni.csv',
    'Poliol-PAni.csv',
    'Pani-NMP.csv',
    'pu-p2.csv'
]
names = {
    'Grafite-p1-pubgd.csv': 'PU + 4% Grafite (bkgnd PU)',
    'Misto-p2-pubgd.csv': 'PU + 2% Grafite + 2% PAni (bkgnd PU)',
    'Pani-p2-pubgd.csv': 'PU + 4% PAni (bkgnd PU)',
    'Grafite-p1.csv': 'PU + 4% Grafite',
    'Misto-p1.csv': 'PU + 2% Grafite + 2% PAni',
    'Pani-p1.csv': 'PU + 4% PAni',
    'Iso-PAni.csv': 'Mistura líquida Isocianato-PAni',
    'Iso-PAni_quente.csv': 'Mistura líquida Isocianato-PAni',
    'Poliol-PAni_quente.csv': 'Mistura líquida Poliol-PAni',
    'Poliol.csv': 'Poliol líquido',
    'Isocianato.csv': 'Isocianato líquido',
    'PAni-ATR-Absor-11-s4.csv': 'PAni ATR',
    'Poliol-PAni.csv': 'Mistura líquida Poliol-PAni',
    'Pani-NMP.csv': 'PAni + NMP',
    'pu-p2.csv': 'PU Puro'
}

In [38]:
for item in list(frames.keys()):
    if item in excluidos:
        frames.pop(item)

In [39]:
fig = go.Figure()
for k,(key, frame) in enumerate(sorted(frames.items(), key=lambda item: names[item[0]])):
    df = frame
    fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'].apply(lambda x: x+k)/2, name = names[key], visible = True))
fig.update_layout(xaxis_range = [4000,500], yaxis_range = [0,2.5],
                  xaxis_title_text = 'Número de onda (1/cm)',
                  yaxis_title_text = 'Transmitância (u.a.)',
                  title_text = 'Espectros de Infra-vermelho',
                  yaxis = dict(tickvals = []),
                  font_size = 16,
                  legend={'traceorder':'reversed'}
                 )
fig

In [12]:
fig.update_layout(newshape_line_color='grey',
                  newshape_line_dash='dash',
                  newshape_line_width=1,
                 )
fig.write_html("ftir.html", config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})

In [13]:
fig = make_subplots(rows = 1, cols = 2,subplot_titles=("Isocianato", "Poliol"), shared_yaxes=True)
df = frames['Iso.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'green', name = 'Puro', legendgroup = 'P'), row = 1, col = 1)
df = frames['Iso-PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'red', name = 'Mistura', legendgroup = 'M'), row = 1, col = 1)
df = frames['TDI-PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'orange', name = 'Mistura aquecida', legendgroup = 'MA'), row = 1, col = 1)
df = frames['PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'blue', name = 'PAni', legendgroup = 'PA'), row = 1, col = 1)
df = frames['Poliol.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'green', showlegend = False, legendgroup = 'P'), row = 1, col = 2)
df = frames['Poliol-PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'red', showlegend = False, legendgroup = 'M'), row = 1, col = 2)
df = frames['PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'blue', showlegend = False, legendgroup = 'PA'), row = 1, col = 2)
df = frames['Poliol-PAni2.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'orange', showlegend = False, legendgroup = 'MA'), row = 1, col = 2)
fig.update_yaxes(showticklabels = False, title_text = 'Transmitância')
fig.update_xaxes(title_text = 'Número de Onda', range = [4000,500])
fig.write_html("ftir1.html")

KeyError: 'Iso.csv'

In [ ]:
fig

In [ ]:
fig = go.Figure()
df = frames['Natural.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], name = 'Grafite Natural'))
df = frames['EG-1000.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], name = 'EG'))
df = frames['NP-6.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance']-70, name = 'NPG'))
fig.update_yaxes(showticklabels = False, title_text = 'Transmitância (U.A.)')
fig.update_xaxes(title_text = 'Número de Onda (1/cm)', range = [4000,500])
fig.update_layout(font_size = 20, template = 'seaborn')
fig.update_traces(line_width = 1)
fig.write_html('ftir2.html')

In [ ]:
fig

## Picos

~3400: Água

1560 - 1600: N-H

In [ ]:
df = frames['Natural.csv']
df['Kind']='Natural'
dfe = frames['EG-1000.csv']
dfe['Kind']='Expandido'
dfn = frames['NP-6.csv']
dfn['Kind']='Nanoplaquetas'

In [ ]:
df = df.merge(dfe, how='outer')
df = df.merge(dfn, how='outer')

In [ ]:
mask = (df['Kind']=='Expandido') | (df['Kind']=='Nanoplaquetas')
df.loc[mask,'%Transmitance'] = df.loc[mask,'%Transmitance']-100
fig = px.line_3d(df, y = 'Wavenumber', z = '%Transmitance', x = 'Kind', color='Kind')

In [ ]:
fig.update_layout(scene_yaxis_range = [4000,400],
                  scene_xaxis_visible = False,
                  scene_yaxis_title_text = 'Número de onda (1/cm)',
                  scene_zaxis_title_text = 'Transmitância (%)',
                  legend_title_text = 'Tipo',
                  scene_yaxis_showgrid = True,
                  scene_zaxis_showgrid = False,
                  scene_zaxis_showticklabels = False,
                  scene_yaxis_showbackground = False,
                  scene_zaxis_showbackground = True,
                 )
fig.write_html('ftir3.html')

In [ ]:
fig